## 1. Taxation data

As Matla is missing in primary-energy-source.csv, it can't be shown in the chart. Therefore, its necessary to substract this value from other averages and totals. 

In [1]:
import pandas as pd

EU_COUNTRIES = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark",
                "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Malta",
                "Italy", "Latvia", "Lithuania", "Luxembourg", "Netherlands", "Poland",
                "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden", "EU 27"]

# Rename columns, filter rows, and subtract Malta's tax from the EU-27
tax = pd.read_csv('data/taxation.csv')
tax = tax[["geo", "TIME_PERIOD", "OBS_VALUE"]]
tax = tax.rename(columns={"geo": "Country", "TIME_PERIOD": "Year", "OBS_VALUE": "Tax"})
tax["Country"] = tax["Country"].replace("European Union - 27 countries (from 2020)", "EU 27")
tax = tax[tax["Country"].isin(EU_COUNTRIES)]
tax = tax[tax["Year"].between(2015, 2022)]
tax.loc[tax["Country"] == "EU 27", "Tax"] = tax.loc[tax["Country"] == "EU 27", "Tax"]\
     - tax.loc[tax["Country"] == "Malta", "Tax"].values[0]
tax = tax[tax["Country"] != "Malta"]
tax

,Country,Year,Tax
0,Austria,2015,8203.70
1,Austria,2016,8384.12
2,Austria,2017,8844.69
3,Austria,2018,8784.06
4,Austria,2019,9070.67
...,...,...,...
262,Slovakia,2018,2226.20
263,Slovakia,2019,2361.53
264,Slovakia,2020,2294.29
265,Slovakia,2021,2382.69


In [2]:
# values used in the .js
print(tax[tax["Country"] == "EU 27"].to_string(float_format="{:.2f}".format))

   Country  Year       Tax
88   EU 27  2015 299534.26
89   EU 27  2016 310183.22
90   EU 27  2017 319291.46
91   EU 27  2018 325976.75
92   EU 27  2019 330579.38
93   EU 27  2020 301423.52
94   EU 27  2021 329949.98
95   EU 27  2022 316924.05


## 2. GDP data

In [3]:
# Same strategy
gdp = pd.read_csv('data/gdp.csv')
gdp = gdp[["geo", "TIME_PERIOD", "OBS_VALUE"]]
gdp = gdp.rename(columns={"geo": "Country", "TIME_PERIOD": "Year", "OBS_VALUE": "GDP"})
gdp["Country"] = gdp["Country"].replace("European Union - 27 countries (from 2020)", "EU 27")
gdp = gdp[gdp["Country"].isin(EU_COUNTRIES)]
gdp = gdp[gdp["Year"].between(2015, 2022)]
gdp.loc[gdp["Country"] == "EU 27", "GDP"] = gdp.loc[gdp["Country"] == "EU 27", "GDP"] \
      - gdp.loc[gdp["Country"] == "Malta", "GDP"].values[0]
gdp = gdp[gdp["Country"] != "Malta"]
gdp

,Country,Year,GDP
39,Austria,2015,342083.5
40,Austria,2016,355665.6
41,Austria,2017,367294.9
42,Austria,2018,383234.3
43,Austria,2019,395706.8
...,...,...,...
995,Slovakia,2018,90275.9
996,Slovakia,2019,94547.5
997,Slovakia,2020,94320.6
998,Slovakia,2021,101960.0


## 3. Emissions data

In [4]:
emissions = pd.read_csv('data/greenhouse-gas-emissions.csv')
emissions = emissions.rename(columns={"Entity": "Country", "Annual greenhouse gas emissions in CO₂ equivalents": "Emissions"})
emissions = emissions[emissions["Year"].between(2015, 2022)]
emissions["Country"] = emissions["Country"].replace("European Union (27)", "EU 27")
emissions = emissions[emissions["Country"].isin(EU_COUNTRIES)]
emissions.loc[emissions["Country"] == "EU 27", "Emissions"] = emissions.loc[emissions["Country"] == "EU 27", "Emissions"] \
      - emissions.loc[emissions["Country"] == "Malta", "Emissions"].values[0]
emissions = emissions[emissions["Country"] != "Malta"]
emissions

,Country,Year,Emissions
2253,Austria,2015,75462370.0
2254,Austria,2016,76328810.0
2255,Austria,2017,78939010.0
2256,Austria,2018,76058420.0
2257,Austria,2019,77420050.0
...,...,...,...
32358,Sweden,2018,74144260.0
32359,Sweden,2019,72984104.0
32360,Sweden,2020,69069270.0
32361,Sweden,2021,72091630.0


## 4. Sources data

In [5]:
# Same strategy but in this case "Clean share" is created as percentage of total emissions
sources = pd.read_csv('data/primary-energy-source.csv')
sources = sources.fillna(0)
sources = sources[sources["Year"].between(2015, 2022)]
sources = sources.drop(columns=['Code'])
sources.columns = ['Entity', 'Year', 'Coal', 'Oil', 'Gas', 'Nuclear', 'Hydro', 'Wind', 'Solar', 'Other renewables']
sources = sources.rename(columns={"Entity": "Country"})
sources = sources.replace("European Union (27)", "EU 27")
sources = sources[sources["Country"].isin(EU_COUNTRIES)]
sources["Total"] = sources.iloc[:, 2:].sum(axis=1)
clean_sources = ["Nuclear", "Hydro", "Wind", "Solar", "Other renewables"]
sources["Clean share"] = sources[clean_sources].sum(axis=1) / sources["Total"] * 100
sources = sources[["Country", "Year", "Clean share"]]
sources = sources.round(2)
sources

,Country,Year,Clean share
522,Austria,2015,33.37
523,Austria,2016,34.45
524,Austria,2017,33.60
525,Austria,2018,33.52
526,Austria,2019,34.68
...,...,...,...
5965,Sweden,2018,70.62
5966,Sweden,2019,70.96
5967,Sweden,2020,72.26
5968,Sweden,2021,71.47


## 5. Votes in EU Parliament

- It was recently updated, but is still available at: https://www.europarl.europa.eu/topics/en/article/20180126STO94114/2024-2029-european-parliament-how-many-meps-per-country

In [6]:
votes22 = {
    "Germany":  96, "France": 79, "Italy": 76, "Spain": 59, "Poland": 52,
    "Romania": 33, "Netherlands": 29, "Belgium": 21, "Czechia": 21,
    "Greece": 21, "Hungary": 21, "Portugal": 21, "Sweden": 21, "Austria": 19,
    "Bulgaria": 17, "Denmark": 14, "Finland": 14, "Croatia": 14, "Ireland": 14,
    "Slovakia": 12, "Lithuania": 11, "Latvia": 8, "Slovenia": 7, "Estonia": 6,
    "Cyprus": 6, "Luxembourg": 6, "EU 27": 705
}

## 6. All data by country

In [ ]:
df = (
    tax.merge(gdp, on=["Country", "Year"])
       .merge(emissions, on=["Country", "Year"])
       .merge(sources, on=["Country", "Year"])
)

# EU values for each year
eu_emissions = df[df["Country"] == "EU 27"].set_index("Year")["Emissions"].to_dict()
eu_taxes = df[df["Country"] == "EU 27"].set_index("Year")["Tax"].to_dict()

# Create cols
df["Tax GDP"] = (df["Tax"] / df["GDP"] * 100)
df["Emissions EU share"] = df["Emissions"] / df["Year"].map(eu_emissions) * 100
df["Tax EU share"] = df["Tax"] / df["Year"].map(eu_taxes) * 100
df["Total amount"] = (df["Emissions EU share"] / 100 * df["Year"].map(eu_taxes)) - df["Tax"]
df["Total amount"] = df["Total amount"] / 1000
df["Balance sheet"] = (df["Emissions EU share"]- df["Tax EU share"])
df["Votes"] = df["Country"].map(votes22)
df["Votes perc"] = df["Votes"] / votes22["EU 27"] * 100

# & filter
df_filtered = df[(df["Year"] >= 2015) & (df["Year"] <= 2022)]
df["Stacked Amount 2015"] = df_filtered.groupby("Country")["Total amount"].cumsum()
df = df[df["Country"] != "EU 27"]
df = df.drop(columns=["GDP", "Tax", "Emissions", "Votes"])
df = df.round(1)
df

,Country,Year,Clean share,Tax GDP,Emissions EU share,Tax EU share,Total amount,Balance sheet,Votes perc,Stacked Amount 2015
0,Austria,2015,33.4,2.4,2.0,2.7,-2.2,-0.7,2.7,-2.2
1,Austria,2016,34.4,2.4,2.0,2.7,-2.0,-0.7,2.7,-4.2
2,Austria,2017,33.6,2.4,2.1,2.8,-2.1,-0.7,2.7,-6.3
3,Austria,2018,33.5,2.3,2.1,2.7,-2.1,-0.6,2.7,-8.4
4,Austria,2019,34.7,2.3,2.2,2.7,-1.8,-0.6,2.7,-10.3
...,...,...,...,...,...,...,...,...,...,...
211,Slovakia,2018,28.4,2.5,1.3,0.7,1.9,0.6,1.7,9.9
212,Slovakia,2019,31.2,2.5,1.1,0.7,1.4,0.4,1.7,11.3
213,Slovakia,2020,32.3,2.4,1.1,0.8,0.9,0.3,1.7,12.2
214,Slovakia,2021,30.2,2.3,1.1,0.7,1.1,0.3,1.7,13.3


In [ ]:
# print again to avoid server (".js style")
output = "\t\tconst contriesData  = [\n"
output += ",\n".join(
    f"\t\t\t{{ Country: \"{row['Country']}\", Year: {row['Year']}, CleanShare: {row['Clean share']}, TaxGDP: {row['Tax GDP']}, EmissionsEUShare: {row['Emissions EU share']}, TaxEUShare: {row['Tax EU share']}, TotalAmount: {row['Total amount']}, BalanceSheet: {row['Balance sheet']}, VotesPerc: {row['Votes perc']}, StackedAmount2015: {row['Stacked Amount 2015']} }}"
    for row in df.to_dict(orient="records")
)
output += "\n\t\t];"
print(output)

		const contriesData  = [
			{ Country: "Austria", Year: 2015, CleanShare: 33.4, TaxGDP: 2.4, EmissionsEUShare: 2.0, TaxEUShare: 2.7, TotalAmount: -2.2, BalanceSheet: -0.7, VotesPerc: 2.7, StackedAmount2015: -2.2 },
			{ Country: "Austria", Year: 2016, CleanShare: 34.4, TaxGDP: 2.4, EmissionsEUShare: 2.0, TaxEUShare: 2.7, TotalAmount: -2.0, BalanceSheet: -0.7, VotesPerc: 2.7, StackedAmount2015: -4.2 },
			{ Country: "Austria", Year: 2017, CleanShare: 33.6, TaxGDP: 2.4, EmissionsEUShare: 2.1, TaxEUShare: 2.8, TotalAmount: -2.1, BalanceSheet: -0.7, VotesPerc: 2.7, StackedAmount2015: -6.3 },
			{ Country: "Austria", Year: 2018, CleanShare: 33.5, TaxGDP: 2.3, EmissionsEUShare: 2.1, TaxEUShare: 2.7, TotalAmount: -2.1, BalanceSheet: -0.6, VotesPerc: 2.7, StackedAmount2015: -8.4 },
			{ Country: "Austria", Year: 2019, CleanShare: 34.7, TaxGDP: 2.3, EmissionsEUShare: 2.2, TaxEUShare: 2.7, TotalAmount: -1.8, BalanceSheet: -0.6, VotesPerc: 2.7, StackedAmount2015: -10.3 },
			{ Country: "Austria"